In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
import os
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

In [ ]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')

In [ ]:
print("Training dataframe shape:", train_df.shape)
print("Test dataframe shape:", test_df.shape)

In [ ]:
train_df = train_df.fillna(0)
train_df.tail()

In [ ]:
one_df = train_df[train_df['Target'] == 1]
two_df = train_df[train_df['Target'] == 2]
three_df = train_df[train_df['Target'] == 3]
four_df = train_df[train_df['Target'] == 4]
print(len(one_df), len(two_df), len(three_df), len(four_df))

In [ ]:
# SMOTE
non_inputs = ['Id', 'Target']
smote = SMOTE(kind = "regular")

x_resampled = train_df[[c for c in train_df if c not in non_inputs]]
x_resampled = x_resampled[[c for c in x_resampled if x_resampled[c].dtype != 'O']]
y_resampled = train_df[['Target']]
x_res, y_res = smote.fit_sample(x_resampled, y_resampled)

train_y = pd.DataFrame({'Target':y_res[:]})
train_x = pd.DataFrame(x_res, columns = x_resampled.columns)

In [ ]:
## Brute force oversampling
"""
one_df = one_df.sample(len(four_df), replace = True)
two_df = two_df.sample(len(four_df), replace = True)
three_df = three_df.sample(len(four_df), replace = True)
four_df = four_df.sample(len(four_df), replace = True)
print(len(one_df), len(two_df), len(three_df), len(four_df))

# new_training_df = pd.concat([one_df, two_df, three_df, four_df], axis = 0)
# new_training_df.head()

# train_x = new_training_df[[c for c in new_training_df if c not in non_inputs]]
# train_y = new_training_df[['Target']]

# train_x = train_x[[c for c in train_x if train_x[c].dtype != 'O']]
# train_y = train_y[[c for c in train_y if train_y[c].dtype != 'O']]
"""

In [ ]:
clf = RandomForestClassifier(max_depth=12, random_state=0, n_estimators=15)
clf.fit(train_x, train_y)

In [ ]:
predictions = list(clf.predict(train_x))
true = train_y['Target'].tolist()
print(classification_report(true, predictions))

In [ ]:
importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
import_features = []
for f in range(train_x.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
    import_features.append(indices[f])

# Plot the feature importances of the forest
plt.figure()
plt.figure(figsize=(20,20))
plt.title("Feature importances")
plt.bar(range(train_x.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(train_x.shape[1]), indices)
plt.xlim([-1, train_x.shape[1]])
plt.show()

In [ ]:
test_df = test_df.fillna(0)

In [ ]:
test_x = test_df[[c for c in test_df if c not in non_inputs]]
test_x.head()

In [ ]:
test_x = test_x[[c for c in test_x if test_x[c].dtype != 'O']]
test_x.head()

In [ ]:
Counter(clf.predict(test_x))

In [ ]:
ids = test_df['Id'].tolist()
targets = list(clf.predict(test_x))

In [ ]:
submission = pd.DataFrame(list(zip(ids, targets)), columns=['Id', 'Target'])
submission.to_csv('sample_submission.csv', index=False)